<a href="https://colab.research.google.com/github/ahembal/MetinMadenciligiLec/blob/main/Homework-01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ls

 Baskaldiran-Insan-Albert-Camus.txt   RNNsStillCool.ipynb   Untitled1.ipynb
'Copy of word_embeddings.ipynb'       Untitled0.ipynb


In [ ]:
cd ..

/content


In [ ]:
cd drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
text = """Hello world this is my notebook, 
and i am very happy to be here where are you and i miss you too much 
i want to see you as soon as possible."""
text2 = "abcde"
text3 = "Hello world this is my notebook"

In [ ]:
import re
import random
import math

In [ ]:
def tokenize(text):
  return re.findall(r"[\w]+|[^\s\w]", text)

In [ ]:
lst = [1,2,3,4,5]
lst[1:1]

[]

In [ ]:
def ctokenize(text):
  return [i for i in text]
 

In [ ]:
ctokenize(text2)

['a', 'b', 'c', 'd', 'e']

In [ ]:
def ngrams(n, text):
    ''' Returns the ngrams of the text as tuples where the first element is
        the length-n context and the second is the character '''
    padded_tokens = []
    if type(text) != str:
      text = "".join(text)
    padded_tokens = "~"*(n) + text + "."
    if n == 1:
      lst = [((padded_tokens[i-1]), token) for i, token in enumerate(padded_tokens) if i >= n-1]
      return  lst[1:]
    else:
      lst = [(("".join(padded_tokens[i-n:i])), token) for i, token in enumerate(padded_tokens) if i >= n-1]
      return  lst[1:]


In [ ]:
ngrams(2,text2)

[('~~', 'a'), ('~a', 'b'), ('ab', 'c'), ('bc', 'd'), ('cd', 'e'), ('de', '.')]

In [ ]:
product = 0
for context, token in ngrams(2, "tokeniz"):
  print(context,token)
  print(n.prob(context,token))
  #product += math.log(n.prob(context, token))
#product

~~ t
0.5
~t o
1.0
to k
0.25
ok e
0.5
ke n
1.0
en i
1.0
ni z
1.0
iz .
0.0


In [ ]:
math.log(1)
math.exp(0)

1.0

In [ ]:
n = NgramModel(1,0)

In [ ]:
n.update('abcd')

In [ ]:
n.update('abab')

In [ ]:
n.perplexity('abcd')

0.8027415617602307

In [ ]:
n.perplexity('abcda')

0.8326831776556043

In [ ]:
n.update("tokenize")
n.update(text)

In [ ]:
n.get_vocab()

dict_keys(['~~', '~H', 'He', 'el', 'll', 'lo', 'o ', ' w', 'wo', 'or', 'rl', 'ld', 'd ', ' t', 'th', 'hi', 'is', 's ', ' i', ' m', 'my', 'y ', ' n', 'no', 'ot', 'te', 'eb', 'bo', 'oo', 'ok', 'k,', ', ', ' \n', '\na', 'an', 'nd', 'i ', ' a', 'am', 'm ', ' v', 've', 'er', 'ry', ' h', 'ha', 'ap', 'pp', 'py', 'to', ' b', 'be', 'e ', 'he', 're', 'wh', 'ar', ' y', 'yo', 'ou', 'u ', 'mi', 'ss', 'mu', 'uc', 'ch', 'h ', '\ni', 'wa', 'nt', 't ', ' s', 'se', 'ee', 'as', 'so', 'on', 'n ', ' p', 'po', 'os', 'si', 'ib', 'bl', 'le', 'e.'])

In [ ]:
n.perplexity("token")

1.5874010519681994

In [ ]:
n.prob(context, token)

2

In [ ]:
context = "".join(["~"] * (2))
context

'~~'

In [ ]:
n.random_text(22)

'Hello be you toon ant '

### kapat burayi

In [ ]:
class NgramModel():

    def __init__(self, n, k):
        self.n = n
        self.k = k
        self.context_dic = {}
        self.context_count_dic = {}
        return

    def get_vocab(self):
      return self.context_count_dic.keys()

    def update(self, sentence):
        for context, token in ngrams(self.n, sentence):
            # keep count
            if context in self.context_count_dic:
                self.context_count_dic[context] += 1
            else:
                self.context_count_dic[context] = 1

            # insert data
            if context in self.context_dic:
                token_dic = self.context_dic.get(context)
                if token in token_dic:
                    token_dic[token] += 1
                else:
                    token_dic[token] = 1
            else:
                self.context_dic[context] = {token: 1}
        return


    def prob(self, context, token):
        if context in self.context_dic:
            token_dic = self.context_dic[context]
            if token in token_dic:
                return float(token_dic[token]) / self.context_count_dic[context]
            else:
                return 0.0
        else:
            return 0.0


    def random_token(self, context):
        r = random.random()

        if context in self.context_dic:
            denominator = self.context_count_dic[context]
            token_dic = self.context_dic[context]
            sorted_keys = sorted(token_dic.keys())

            for i, token in enumerate(sorted_keys):
                minus_i_sum = sum([token_dic[k] for k in sorted_keys[:i]])
                if float(minus_i_sum)/denominator <= r < float(minus_i_sum + token_dic[sorted_keys[i]])/denominator:
                    return token

        else:
            return None

    def random_text(self, token_count):

        if self.n != 1:
            context = "".join(["~"] * (self.n))
            generated = []

            for __ in range(token_count):
                token = self.random_token(context)
                generated.append(token)

                if token != ".":
                    context = context[1:] + (token)
                else:
                    context = "".join(["~"] * (self.n))

            return "".join(generated)
        else:
            return "".join(self.random_token(random.choice(list(self.get_vocab()))) for _ in range(token_count)) 

    def perplexity(self, sentence):
        product = 0

        for context, token in ngrams(self.n, sentence):
          if self.prob(context,token) != 0:
            product += math.log(self.prob(context, token))
        return (1/math.exp(product)) ** (float(1)/(len(sentence)+1))


In [ ]:
def create_ngram_model(n, path):
    model = NgramModel(n)

    with open(path, 'r+') as f:
        for line in f:
            model.update(line)

    return model

In [ ]:
mdl = create_ngram_model(NgramModel,"Baskaldiran-Insan-Albert-Camus.txt",5,0)

In [ ]:
def create_ngram_model(model_class, path, n=2, k=0):
    ''' Creates and returns a new n-gram model trained on the city names
        found in the path file '''
    model = model_class(n, k)
    with open(path, encoding='utf-8', errors='ignore') as f:
        model.update(f.read())
    return model

In [ ]:
import numpy as np
test = np.array([1,2,3])  

In [ ]:
test.shape

(3,)